In [ ]:
import sys
print(sys.version)
print(*sys.path, sep='\n')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision as tv

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

num_workers = os.cpu_count()
print(num_workers)

cuda:0
6


In [4]:
# parameters for trained CNN
root = os.path.join('.', 'dataset_root')
# mean, std = [0.13066046], [0.30150425] # based on training set
batch_size = 64
lr = 0.001
epochs = 30

In [5]:
def imshow(tensor_grid, mean=0., std=1., title=None):
    assert isinstance(tensor_grid, torch.Tensor)
    assert len(tensor_grid.size()) == 4, \
        f'For a batch of images only, {tensor_grid.size()} '
    
    tensor_grid = tv.utils.make_grid(tensor_grid)
    grid = tensor_grid.numpy().transpose((1,2,0))
    grid = std * grid + mean
    grid = np.clip(grid, 0, 1)
    plt.imshow(grid)
    
    if title is not None:
        plt.title(title)
        
    plt.pause(0.001)

In [6]:
# prepare data
# foolbox model expects raw numpy array as image
transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
#         tv.transforms.Normalize(mean, std)
])
train_dataset = tv.datasets.MNIST(
    root,
    train=True, 
    download=True,
    transform=transform)
test_dataset = tv.datasets.MNIST(
    root,
    train=False, 
    download=True,
    transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(int(((28 - (3-1)) / 2)**2 * 8), 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [8]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0.
    corrects = 0
    
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        batch_size = x.size(0)
        
        optimizer.zero_grad()
        output = model(x)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()

        # for display
        total_loss += loss.item() * batch_size
        preds = output.max(1, keepdim=True)[1]
        corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(train_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [9]:
def validate(model, loader):
    model.eval()
    total_loss = 0.
    corrects = 0
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            output = model(x)
            loss = F.nll_loss(output, y)
            total_loss += loss.item() * batch_size
            preds = output.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(test_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [10]:
# NOTE: NO GPU AT SCHOOL!
model = Net()
model_softmax = torch.nn.Sequential(
    model,
    torch.nn.LogSoftmax(dim=1)
)
model_softmax.to(device)
optimizer = torch.optim.SGD(model_softmax.parameters(), lr=lr)

In [12]:
since = time.time()
for epoch in range(epochs):
    start = time.time()
    tr_loss, tr_acc = train(model_softmax, train_loader, optimizer)
    va_loss, va_acc = validate(model_softmax, test_loader)
    
    time_elapsed = time.time() - start
    print(('[{:2d}] {:.0f}m {:.1f}s Train Loss: {:.4f} Accuracy: {:.4f}%, ' +
        'Test Loss: {:.4f} Accuracy: {:.4f}%').format(
            epoch+1, time_elapsed // 60, time_elapsed % 60,
            tr_loss, tr_acc*100.,
            va_loss, va_acc*100.))
    
    # exit early
#     if va_acc > 0.98:
#         break
    
time_elapsed = time.time() - since
print('Total run time: {:.0f}m {:.1f}s'.format(
    time_elapsed // 60,
    time_elapsed % 60))

[ 1] 0m 2.1s Train Loss: 2.2541 Accuracy: 32.0733%, Test Loss: 2.1774 Accuracy: 56.1800%
[ 2] 0m 2.0s Train Loss: 2.0411 Accuracy: 58.3683%, Test Loss: 1.8069 Accuracy: 70.1900%
[ 3] 0m 2.0s Train Loss: 1.5100 Accuracy: 70.7017%, Test Loss: 1.1127 Accuracy: 79.1700%
[ 4] 0m 2.0s Train Loss: 0.9823 Accuracy: 79.5967%, Test Loss: 0.7097 Accuracy: 84.0700%
[ 5] 0m 2.0s Train Loss: 0.7287 Accuracy: 83.5467%, Test Loss: 0.5413 Accuracy: 86.5500%
[ 6] 0m 2.3s Train Loss: 0.6090 Accuracy: 85.7550%, Test Loss: 0.4591 Accuracy: 88.0200%
[ 7] 0m 2.2s Train Loss: 0.5392 Accuracy: 87.0733%, Test Loss: 0.4111 Accuracy: 88.9300%
[ 8] 0m 2.1s Train Loss: 0.5000 Accuracy: 87.7367%, Test Loss: 0.3813 Accuracy: 89.3900%
[ 9] 0m 2.3s Train Loss: 0.4654 Accuracy: 88.3217%, Test Loss: 0.3616 Accuracy: 89.8000%
[10] 0m 2.1s Train Loss: 0.4477 Accuracy: 88.7600%, Test Loss: 0.3474 Accuracy: 89.9900%
[11] 0m 2.0s Train Loss: 0.4336 Accuracy: 89.0167%, Test Loss: 0.3374 Accuracy: 90.3800%
[12] 0m 2.1s Train Lo

In [13]:
torch.save(model.state_dict(), 'mnist_simple_nosoftmax.pt')
torch.save(model_softmax.state_dict(), 'mnist_simple_full.pt')